# Advanced LSL Use

We've now taken a look at the basics of LSL, streaming random data points from an outlet to and inlet and logged them to a file.

In practice, LSL use is always associated with an external data source. This could simply be sending data points or markers from external software (like a game engine) or accessing sensor readings with Python and pushing them to an outlet. Sometimes you'll want to use separate computers altogether for sampling and recording to reduce the computational load on a single machine. In this case, LSL's built-in networking is extremely convenient.

![](./images/lsl-structure-p4i.drawio.png)

At this point it's worth emphasizing that we've already learned the general LSL workflow and you should now be ready to use it in practice, no matter the data source you have: writing a sampling solution is very much dependent on what data sources you use and how well they are supported in Python.

In this part, we'll reach outside of LSL and go through a couple of practical examples of uses we've faced when doing research in our group.

## Streaming BITalino Sensor Data

## Sending Markers Over HTTP

Even if your sensing solution [supports LSL](https://labstreaminglayer.readthedocs.io/info/supported_devices.html), when doing HCI research, sometimes your participants will be interacting with software that is not developed with Python (e.g. web applications). While the core of LSL - liblsl - is available for other languages (C and C++ having active development at the time of writing, with Java, C# and MATLAB seemingly in a limbo), there's a more flexible option that is not dependent on language-specific support: the good old [HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP/Overview) request. In essence, your application can do a HTTP request to a Python server, which will then pass a marker to LSL. 

This method is fairly language-agnostic as most commonly used programming languages are able to make HTTP requests, but due to the nature of the protocol, this is only really useful for marker information. For continous data, you might be interested in [WebSockets](https://developer.mozilla.org/en-US/docs/Web/API/WebSockets_API), though it's not nearly as convenient as LSL since you'll need to handle connectivity issues yourself!

Let's make a quick HTTP server with [Flask](https://flask.palletsprojects.com/en/3.0.x/) and have it pass on markers to LSL. Note that you cannot run the server within the notebook; we'll try running it a bit later!

In [ ]:
from flask import Flask, make_response, request
from flask_cors import CORS
from pylsl import StreamInfo, StreamOutlet

# Create the Flask app
app = Flask(__name__)

# Enable cross-origin resource sharing for a specific URL. This only lets in requests from the specified URL.
origin_url = 'your_url_here'
cors = CORS(app, origins=[origin_url])

# Define the stream & outlet
stream = StreamInfo('markers', 'markers', 1, 0, 'string', 'ws-flask-markers')
outlet = StreamOutlet(stream)

# Define a route where markers should be sent
@app.route('/markers', methods = ['POST'])                          # Route where requests should be sent is <server_url>/markers. Only POST requests are accepted.
def receive_marker():
    try:
        req_contents = request.get_json()                           # Parse the contents of the HTTP request (to a JSON object)
        marker = str(req_contents['marker'])                        # We'll put the marker string in a field called "marker" when making the request
        print('Marker received with HTTP:', marker)
        outlet.push_sample([marker])                                # Push the marker to the outlet
        return make_response({'message': 'OK'}, 202)                # HTTP requests have to return at least a status code!
    except Exception as e:
        return make_response({'message': str(e)}, 500)              # If an error happened during processing, return the error message

To test our marker server, let's write a short script that sends [HTTP POST](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods/POST) requests at a steady interval. Again, this won't work quite yet.

In [ ]:
import time
import requests                                             # The requests library can be used in Python to make HTTP requests

while True:
    requests.post(                                          # Construct a HTTP POST request
        'http://127.0.0.1:5000/markers',                    # The URL of our Flask server & marker endpoint
        json = {'marker': 'hello'},                         # The body of our request contains the marker
        headers = {'Content-Type': 'application/json'}      # Set the HTTP Content-Type header so our server accepts the contents
    )

    time.sleep(2)                                           # Sleep for 2 seconds to prevent overwhelming the server

Let's test this in an exercise.

### D. HTTP Markers

[The exercise folder](./exercises/D_HTTP-markers/) contains two scripts; `app.py` is our server script from above, while `marker_gen_http.py` is the request script.

Familiarize yourself with the two scripts, and try running them side-by-side:

```bash
$ flask run                         # This automatically executes the app.py script

...

$ python marker_gen_http.py
```

You should see the server receiving a request with the message "hello" every two seconds, which is then pushed to the LSL network. 

> Note that `app.py` has a different CORS option from the example above. A wildcard (`*`) enables requests from any source, but using one is usually strongly NOT recommended, as it enables requests from any source, leaving your server vulnerable to malicious use. In practice, you'll always want to specify a single origin (like in the example above), but the following task requires the use of a wildcard.

Your next task is to send a request from any other HTTP client to the server and have the server print out the marker from the request alongside code 202. 

The request should:
- **be a `POST` request**
  - ...as opposed to GET, OPTIONS, PUT, DELETE
- **be directed to `http://127.0.0.1:5000/markers`**
  - ...where `http://` specifies our protocol, `127.0.0.1` specifies the address of our server in the local network, `5000` is the port of our server, and `/markers` is our server's marker endpoint.
- **have a `Content-Type` header with the value `application/json`**
- **contain the following JSON `{"marker": "your_marker_text_here"}` in the body of the request**

An easy way to test HTTP requests is by installing the [Thunder Client VSCode extension](https://www.thunderclient.com/). Alternatively, if you're feeling adventurous, you can write a short script in, say, JavaScript, and use [`fetch`](https://developer.mozilla.org/en-US/docs/Web/API/Fetch_API/Using_Fetch).

You only need to run the server script when testing your request. You do not need to modify the server script.

## Group Exercise

Let's cap off the workshop with a group / pair exercise. If you really don't want to work with others and have two computers at your disposal, you can also do this on your own 😄

### E. Streaming Between Devices

The [directory](./exercises/E_Streaming-between-devices/) contains our simple marker generator and logger from part 2 / exercise A. Coordinate with a friend (one of you runs the generator and the other the logger) so that markers are sent from one computer to another.

- Define the StreamInfo for the generator script on the sender's computer.
- Define the source id for the logger script on the receiver's computer
- The sent marker can be anything you want
- Make sure you are both connected to the same network
- Make sure the stream identifier is the same for the both of you

**Bonus:** Now do the same with two senders and one receiver!

## Conclusion

Congratulations! 🎉🎉🎉

You've somehow managed to wade all the way here! You're now all set to develop your own LSL solutions for your and/or others research needs. Happy debugging!

